In [ ]:
# import os
from collections import OrderedDict
from datetime import datetime, date
import numpy as np
import pandas as pd

In [63]:
# Globals
HOME_HANDICAP = 4
NUM_TEAMS = 30
DEFAULT_CONFIDENCE = float((NUM_TEAMS + 1) / 2)

In [ ]:
year = 2015
sched_f = "csvs\\nba_season_scores_{0}.csv".format(year)
wt_f = "csvs\\nba_season_win_totals_{0}.csv".format(year)

In [ ]:
sched_df = pd.read_csv(sched_f)
# sched_df.sort_values(by=["game_id","game_date"]).head()

In [ ]:
wt_df = pd.read_csv(wt_f) #, index_col=False)

In [ ]:
tms_all = wt_df['team_abbrv']
# tms_all = tms_all[:5]

In [ ]:
tm_results_dict_keys = ['team_abbrv', 'sum_win', 'sum_loss']
tm_results_dict = OrderedDict((key, []) for key in tm_results_dict_keys)


In [ ]:
model_dict_keys = ['game_pick', 'game_pick_correct_flag', 'game_win_flag',
                       'game_loss_flag', 'sum_win', 'sum_loss', 'sum_error', 'wt_pick_correct_flag']
model_dict = OrderedDict((key, []) for key in model_dict_keys)

In [ ]:
# for t in tms_all:
time_start = datetime.now()

t = tm_all[0]

In [ ]:
# Need tmo convert type from pandas.series to float.
tm_wt = floatm(wtm_df.loc[wtm_df['team_abbrv'] == tm]['win_total'])
tm_df = sched_df.loc[sched_df['team_schedule'] == tm]

sum_win_actmual = tm_df['wins_to_date'].iloc[-1]
actmual_sum_loss_num = tm_df['losses_to_datm'].iloc[-1]

sum_win_model = sum(model_dict['game_win_flag'])
sum_loss_model = sum(model_dict['game_loss_flag'])

In [ ]:
# Use preseason win total to determine winner.
def get_winner_simple1(tm1, tm2, home_flag, tm1_wt, tm2_wt):
    if (tm1_wt > tm2_wt) or (tm1_wt == tm2_wt and home_flag):
        return tm1
    else:
        return tm2

# Use win total to date to determine winner.
def get_winner_simple2(tm1, tm2, home_flag, tm1_wins_to_date, game_date, scores_df):
    tm2_df = scores_df.loc[scores_df['team_schedule'] == tm2]
    tm2_wins_to_date = int(
        tm2_df.loc[tm2_df['game_date'] == game_date]['wins_to_date'])

    if (tm1_wins_to_date > tm2_wins_to_date) or (tm1_wins_to_date == tm2_wins_to_date and home_flag):
        return tm1
    else:
        return tm2


In [ ]:
def get_model_tm_mu(person, wt_df, tm, tm_wt, tm_sched, home_flag):
    try:
        model_wt_pick_flag = str(
            wt_df.loc[wt_df['team_abbrv'] == tm]['{0}_win_total_pick_flag'.format(person)])
        # account for case in which number (in addition to 'o' or 'u') is gven for win total
        # model_wt_pick_num = int(wt_df.loc[wt_df['team_abbrv'] == t]['model_win_total_pick_num'])

        if model_wt_pick_flag == 'o':
            # use number above team's win total if 'over'
            mu = int(round(tm_wt+1))
        else:
            # use number below team's win total if 'under'
            mu = int(round(tm_wt))

    except Exception as e:
        # print e
        mu = float(tm_wt)

    # tm == tm_sched if it is the tm that is currently being analyzed.
    if tm == tm_sched:
        # print "Debugging: home_flag type is {0}".format(type(home_flag))
        if home_flag == 1:
            mu += float(HOME_HANDICAP/2)
        elif home_flag == 0:
            mu -= float(HOME_HANDICAP/2)
    else:
        if home_flag == 0:
            mu += float(HOME_HANDICAP/2)
        elif home_flag == 1:
            mu -= float(HOME_HANDICAP/2)

    return mu


def get_model_tm_sigma(wt_df, tm, num_teams):
    # DEFAULT_CONFIDENCE = float((NUM_TEAMS + 1) / 2)
    try:
        model_wt_confidence = int(
            wt_df.loc[wt_df['team_abbrv'] == tm]['model_win_total_confidence'])

        # account for 'model_win_total_confidence' = None in older
        # 'nba-season-win-totals-yyyy.csv' files
        if model_wt_confidence == None:
            model_wt_confidence = DEFAULT_CONFIDENCE
            # print "No 'model_win_total_confidence' for {0}".format(t)
            # print "Using a default value"

    except Exception as e:
        # print e
        model_wt_confidence = DEFAULT_CONFIDENCE

    return float((num_teams+1)-model_wt_confidence)


def get_model_tm_wt_pick(mu, sigma):
    return sigma * np.random.randn() + mu

In [ ]:
def get_model3_game_winner(this_t, other_t, this_t_wt, other_t_wt, home_flag, wt_df, num_teams, model3_dict2):
    # mean of normal distribution
    this_t_mu_num = get_model3_mu_num(
        wt_df, this_t, this_t_wt, this_t, home_flag)
    other_t_mu_num = get_model3_mu_num(
        wt_df, other_t, other_t_wt, this_t, home_flag)

    # standard deviation of normal distribution
    this_t_sigma_num = get_model3_sigma_num(wt_df, this_t, num_teams)
    other_t_sigma_num = get_model3_sigma_num(wt_df, other_t, num_teams)

    # calculate normal distribution N(mu, sigma^2)
    this_t_wt_pick_num = get_model3_wt_pick_num(
        this_t_mu_num, this_t_sigma_num)
    other_t_wt_pick_num = get_model3_wt_pick_num(
        other_t_mu_num, other_t_sigma_num)

    update_model3_dict2_for_game(model3_dict2, this_t_mu_num, this_t_sigma_num,
                                 this_t_wt_pick_num, this_t, this_t)
    update_model3_dict2_for_game(
        model3_dict2, other_t_mu_num, other_t_sigma_num, other_t_wt_pick_num, other_t, this_t)

    if (this_t_wt_pick_num > other_t_wt_pick_num) or (this_t_wt_pick_num == other_t_wt_pick_num and home_flag):
        return this_t
    else:
        return other_t

In [ ]:
for index, row in tm_df.iterrows():
    home_flag = int(row.loc['home_flag'])
    win_flag = int(row.loc['win_flag'])

    # Identify if tm is home or away.
    if home_flag == 1:
        tm2 = row.loc['away_team']
    else:
        tm2 = row.loc['home_team']

    # need to convert type from pandas.series to float
    tm2_wt = float(
        wt_df.loc[wt_df['team_abbrv'] == tm2]['win_total'])